# AI Agent for BaghChal

* Akash Shrestha <Akash.Shrestha@colostate.edu>
* Prashant K. Thakur <Prashant.Thakur@colostate.edu>

## Introduction

BaghChal is a traditional game originated from Nepal which we grew up playing. The board has 4 tigers (starting at four corners of the board) and 20 goats (placed gradually on the board). The game from tiger perspective is to capture the goats (5 of them) for a win. The goats on the other hand try to stick into a herd and make the tigers immobile. The  pieces are placed on the vertices and the movement are only legal if the vertices are connected through edges. The goats can move from one edge to the next if connecting vertex is empty. While the tiger can move to an adjacent connecting empty vertex or jump over a goat and move to vertex 2 hop away from it along the connecting edges.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Empty_Alquerque_board.svg/300px-Empty_Alquerque_board.svg.png" />

Since this game has some local origination, it is not common to find an artificial agent implemented for this game which motivated us.  Further, Bagh-Chal is an asymmetric game where both players have different move strategies and goal. The state space of Bagh-Chal is very huge which makes it impossible for the agent to search all the states in a limited time. The state space of this game is given below $^{[3]}$:

where,

$S_0$: all the states that can occur during placement phase.

$S_k$: all the states that can occur during sliding phase, with four tigers, (20-k+1) goats and k empty spots on board.

| |# of board images |
|------|------|
| $S_0$ |3,316,529,500 |
| $S_1$ |33,481 |
| $S_2$ | 333,481 |
| $S_3$ | 2,105,695 |
| $S_4$ | 9,469,965 |
| $S_5$ | 32,188,170 |

We wanted to explore the AI agent for this game and wanted to implement different evaluation functions for better preformance.

## Methods

Board games can be modelled as game tree. Each node of the tree denote some state in the game. Edges denote the actions in game. The successors from that node are all possible states the game can go to by taking some actions. The role of an AI agent is to find goal state starting from some starting state, trying to maximize/minimize some objective depending on the domain. For tiger, the objective that we want to maximize is number of goats captured. In general, AI agent performs search in this space, which for BaghChal is big as mentioned in above table. We can use different algorithms like Minimax, Alpha-Beta Pruning etc for solving this search problem.

**Minimax** is a decision making algorithm used in game theory. For minimax algorithm, there are two players - MAX and MIN. MAX and MIN plays alternatively. Using minimax, we explore the future state the game can take from current state and assign values to the leaf nodes. These leaf nodes might be terminal state of the game or some intermediate state where we decide to stop exploring further to cut down the computational overhead. The values that we assign to the leaf nodes are called utility values, which denote the payoff for player playing on that ply. This utility value should be minimized for opponent player and maximized for us to win. At each depth, we minimize if it's opponent's turn and maximize if it's our turn. Finally, we choose the move at current state of a board which gives us a maximum utility in the future.

**Alpha-beta pruning** is a strategy to reduce number of nodes that we visit in minimax algorithm. Minimax algorithm is basically a depth-first search. We already know values of some leaf nodes before we explore other leaf nodes. At every level, we are either maximizing or minimizing. So at each level we are considering only one node out of all siblings at that level to be further evaluated in the upper levels. This way we are doing some unnecessary node exploration in minimax. In order to prevent this, we keep track of best utility found for MAX player as $\alpha$ and best utility found for MIN player as $\beta$ in alpha-beta. So at any nodes if $\alpha>\beta$, we stop exploring further from that level. This way we reduce number of nodes visited in minimax and get to the same result in less time and space. Further, if we explore good states at the beginning, we will end up pruning a lot of tree. This process is called **move ordering**.

### Our approach to building AI agent

We initiated by looking at interfaces and methods of board games that we solved as assignment for this course. Then we began modelling of BaghChal game and modelling of AI agent parallely. Prashant had major contribution towards game modelling. We built BaghChal board in ASCII format and internal representation were python lists for both board state and moves. We created methods for constructing game tree in general. At the same time, Akash contributed towards building agent - methods for game playing with minimax and alpha-beta algorithms. 

After we have board and dummy game playing agent in python, we began exploring possible heuristics for evaluating the board state. We implemented them in python and analysed the behaviour of agent with respect to those functions. We tried different evaluation functions which are discussed later in Results section.

Later on we did analysis with minimax, alpha-beta pruning with and without move ordering. For the latter parts, both of us contributed equally and most of the tasks were accomplished together.

The board model is implemented as `GameBoard` class in `baghchal.py`. Agent related game-playing and minimax, alpha-beta algorithms are implemented in `MiniMax` class of `agent.py`. Evaluation functions are in `evaluation.py`. The description of these classes are given in [BaghChal Game Overview.ipynb](BaghChal%20Game%20Overview.ipynb).

In [1]:
from baghchal import GameBoard
from evaluation import Evaluation
from agent import MiniMax

## Results

The game board and rules are defined in `GameBoard` class in `bagchal.py`. The ASCII representation of the game is shown here:

In [2]:
game = GameBoard()
print(game)

T---o---o---o---T
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
o---o---o---o---o
| \ | / | \ | / |
o---o---o---o---o
| / | \ | / | \ |
T---o---o---o---T



While playing, tigers will move across the connecting edges and there would be many (20 in total number) goats in the board. The intermediate state during gameplay will look like:

In [3]:
game.board = ['T','','','G','G','T','G','G','','','','G','T','G','G','G','G','','G','G','G','G','G','G','T']
print(game)

T---o---o---G---G
| \ | / | \ | / |
T---G---G---o---o
| / | \ | / | \ |
o---G---T---G---G
| \ | / | \ | / |
G---G---o---G---G
| / | \ | / | \ |
G---G---G---G---T



### Evaluation functions

Due to asymmetric nature of game, the board evaulation for tiger and goat are different. The goats tend to stick together in a herd and make the tiger immobile while the tiger approach is to seperate the goat out of its herd and capture them. Meanwhile, tigers need to make sure they are not trapped by the herd and increase their mobility. Based on the nature of steps involved for two opponents, we have defined two heuristic functions for any board position.

* First one is movability of tigers, `tigerMovability(game)`, which denote all possible moves for a tiger player in a given board position.
* Next is distance among goats, `goatDistanceAmong(game)`. Distance between two goats is like manhattan distance but considering the diagonal lines on board. Then distance among goats is defined as sum of distances from one goat to every other goats divided by total number of goats on a board.

#### Evaluation function 1

Linear combination of distance among goats and tiger movability.

We made an AI agent which plays as a tiger: minimax upto depth 3 with evaluation function against dummy goat player which choses random move. In order to make a victorous tiger we select the moves which would increase the tigers mobility and also increase the distance between the goats. 

$$evaluationTiger(game) = goatDistanceAmong(game) + 5*tigerMovability(game)$$

Out of 500 games, number of wins are as below:

<img src="tigerEvaluation1.png" width="220px" height="240px" />

This evaluation function is simple which only helps the tiger to select the moves to have greater mobility while considering the distance between the goats. This gives the tigers some superiority because of the information from the board about the position of goats and its consideration to keep its path clear for greater mobility.

#### Evaluation function 2

We made an AI agent play as goat: minimax upto depth 3 with evaluation function against dummy tiger which chooses a random move. The parameters considered is the inter-goat distance and the mobility of the tigers on a given board. The distance between the goats is marked negatively because the goats are ment to stay in the herd. Similarly, the moves which restrains the mobility of the tigers are highly likely to increase the chance of win for the goats.  

$$evaluationGoat(game)=-1*[goatDistanceAmong(game)+5∗tigerMovability(game)]$$

We played 500 games again and number of win-loss are as below:

<img src="goatEvaluation1.png" width="220px" height="240px" />

This evaluation function for a goat didn't perform so good. While analysing the returns from minimax, we observed that the moves which tend to block tiger were prefered more. This was because of `5*tigerMovability(game)` in above equation. The goat tends to block the tigers mobility even at the risk of sacrificing itself. Even though the tiger moves are random, there is a high probability for a goat to be captured by tiger when it finds an isolated goat closer to tiger despite herd structure. 

#### Evaluation function 3

From above results, we were certain that some informations could be added into the heuristic functions to make it better. Couple of extra informations that we can extract from given board are: number of vulnerable goats and number of goats captured. So the other two heuristics in addition to above mentioned heuristics are defined as:

* Number of vulnerable goats, `vulnerableGoats(game)` denote the number of goats which could be captured by tiger from current state of game.
* Number of goats captured, `deadGoats(game)` denote the total goat loss when we follow certain path in game tree.

The objective for goat player should be to reduce `vulnerableGoats(game)` and `deadGoats(game)`.

The new evaluation function for goat is:
$$evaluationGoat(game)=
-1*[0.5*goatDistanceAmong(game)+5∗tigerMovability(game)+5*vulnerableGoats(game)+5*deadGoats(game)]$$

<img src="goatEvaluation2.png" width="220px" height="240px" />

With these two new information, goat is playing well than before. Now with the above heuristic function the goat would get negative weightage if there are already some captured goats. Since the tiger wins if it can capture 5 goats, this evaluation makes the goat to select the moves which would save the goats from being captured along with restricting the movement of tigers on the board. Similarly, the vulnerable goat counts are also added to select the moves such that the moves which would save another goats from being captured. The approach was to add additional evaluation functions so that the goat can analyze the board position efficiently and the number of wins in the trial games suggests the goats were doing better than other evaluation functions discussed above.

### Moves ordering in Alpha beta pruning

For a tiger player, moves that give us immediate goat capture is comparatively better than just wandering around. So for tiger player, the jumping moves (which tend to capture goat) are weighted higher. Rest of the moves are weighted zero by default. We sort the moves based on this score and visit those moves with higher score with priority.

For a goat player, moves that tend to defend a capture of goat is better move. So from current board state, goat player evaluates possible tiger moves and higher weightage is given to those moves whose destination vertex is to oppose the tiger's jumping move. We sort the moves based on this and visit them with priority.

In [4]:
def sortTigerMoves(moves):
    scoredMoves = []
    for move in moves:
        source = to2D(move[0])
        dest = to2D(move[1])
        # Jumping moves are given high score
        scoredMoves.append( ( move, max(abs(dest[0] - source[0]), abs(dest[1] - source[1])) ) )

    scoredMoves.sort(key=lambda x:x[1], reverse=True)
    return [move[0] for move in scoredMoves]

def sortGoatMoves(moves):
    valuedDest = []
    tigerMoves = valid_tiger_move()
    # Find tiger's jumping moves
    for move in tigerMoves:
        source = to2D(move[0])
        dest = to2D(move[1])
        if max(abs(dest[0] - source[0]), abs(dest[1] - source[1])) > 1:
            valuedDest.append(move[1])

    # Check if dest idx of goat is in tiger's jumping moves
    if len(valuedDest) > 0:
        scoredMoves = []
        for move in moves:
            if move[1] in valuedDest:
                scoredMoves.append((move, 1))
            else:
                scoredMoves.append((move, 0))
        scoredMoves.sort(key=lambda x: x[1], reverse=True)
        return [move[0] for move in scoredMoves]
    else:
        return moves

There can be other move ordering strategies too which we haven't yet explored. We could have considered prioritizing moves that disrupts the herd, or other extra informations which we haven't yet explored.

### Effective branching factor

Branching factor of a node is the total number of nodes produced from that node. All the nodes don't have same number of children, so effective branching factor(`EBF`) is an approximation of branching factor for a search algorithm. We are using EBF measure to compare efficiency of different search algorithms we have used in our game. We want to compare:
1. Minimax search
2. Minimax with alphabeta pruning
3. Minimax with Alphabeta pruning+moves ordering

To compare between those algorithms in terms of EBF, we need different games to follow same path, which doesn't happen because one of the player is playing random moves. So we made AI play 500 games with depth limit 4 with those three algorithms and took average of EBF values. The average of EBF and 95% Confidence Interval out of those 500 game-plays are as shown below:

<img src="moveOrdering.png" width="600px" />

As seen in above chart, alpha-beta pruning with move ordering is performing search with less branching factor. Our move ordering strategy was very simple and yet we observed very good pruning. We could explore more on move ordering for making it more efficient.

## Conclusion

In this project, we created a game playing agent for BaghChal and tried to enhance the performance by working out on some heuristics functions to evaluate the weightage of game board. The game has several parameter that could be considered and those intelligence could have been built in the players. For this project we have considered the mobility of the tiger, distance of goat from one another, number of goat captured, number of vulnerable goats on a board. Because of the limitation on the time, we have to sum up our project with mentioned analysis. Nevertheless, other heuristic function could also have been feasible which provides an intelligence to a piece to position itself in a better position while making a side moves. Additionally, we have implemented only one way of moves sorting to find the illegible move out of several moves for a given board. There is still some possibilities for move ordering that could enhance the performance of the overall game. 

### References

[1] [Russell and Norvig, 2014] Stuart Russell and Peter Norvig, Artificial Intelligence: A Modern Approach, Publisher. 2014.

[2] [CS440 Lecture] Prof. Chuck Anderson Lecture Notes and Lecture Materials.

[3] Jin, Lim Yew, and J. Nievergelt. "Computing Tigers and Goats." ICGA Journal 27.3 (2004): 131-141.


In [5]:
import io
from nbformat import current
import glob
nbfile = glob.glob('*.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[1], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[1], 'is', word_count)

More than one ipynb file. Using the first one.  nbfile= ['BaghChal Game Overview.ipynb', 'Project Report.ipynb']
Word count for file Project Report.ipynb is 2225


/Users/akash/Downloads/Fall17/CS440/env/lib/python3.6/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)
